In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from CustomFAISS import *
import pandas as pd
import numpy as np
import faiss

In [3]:
data = pd.read_csv('embeddings_maths.csv', usecols=[i for i in range(2, 386)]).to_numpy()
data.nbytes / (1024 * 1024) #MB

40.9013671875

In [4]:
train_data = data[:13900, :]
test_data = data[13900:, :]

In [5]:
model_path = "sentence-transformers/all-MiniLM-l6-v2"

model = HuggingFaceEmbeddings(
                            model_name=model_path,
                            model_kwargs={'device': 'cpu'},
                            )

## FAISS Baseline

In [6]:
def recall_n(X, baseline, n):
    assert X.shape == baseline.shape
    assert baseline.shape[1] >= n

    recall_n = 0
    
    for i in range(X.shape[0]):
        for j in range(n):
            if X[i,j] in baseline[i, :n]:
                recall_n += 1
    
    return recall_n / (X.shape[0] * n) * 100

def smetric_9(X, baseline):
    assert X.shape == baseline.shape
    assert baseline.shape[1] >= 9

    smetric = 0

    for i in range(X.shape[0]):
        for j in range(9):
            if baseline[i,j] in X[i, :9]:
                smetric += 9 - j
    
    return smetric / X.shape[0]

In [7]:
#%%timeit
index = faiss.IndexFlatL2(train_data.shape[1])
index.train(train_data)
index.add(train_data)

In [8]:
codes = faiss.vector_to_array(index.codes)
codes.nbytes / (1024 * 1024) #MB

20.361328125

In [9]:
%%timeit
_, closest_baseline = index.search(test_data[0].reshape(1,-1), k=10)

651 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
#%%timeit
_, closest_baseline = index.search(test_data, k=10)

## Custom Flat

In [11]:
%%timeit
index = CustomIndexFlat(train_data.shape[1])
index.train(train_data)
index.add(train_data)

4.62 ms ± 6.59 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
index = CustomIndexFlat(train_data.shape[1])
index.train(train_data)
index.add(train_data)

In [14]:
codes = index.codes
codes.nbytes / (1024 * 1024) #MB

20.361328125

In [13]:
%%timeit
_, closest_customflat = index.search(test_data[0].reshape(1,-1), k=10)

23 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
_, closest_customflat = index.search(test_data, k=10)

51.6 ms ± 721 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
_, closest_customflat = index.search(test_data, k=10)

In [16]:
print(f'smetric@9 = {smetric_9(closest_customflat, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_customflat, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_customflat, closest_baseline, 1):.1f}')

smetric@9 = 45.00
recall@10 = 100.0
recall@1 = 100.0


## FAISS PQ

In [32]:
#%%timeit
index_pq = faiss.IndexPQ(train_data.shape[1], 128, 8)
index_pq.train(train_data)
index_pq.add(train_data)

In [33]:
codes = faiss.vector_to_array(index_pq.codes)
codes.nbytes / (1024 * 1024) #MB

1.69677734375

In [34]:
%%timeit
_, closest_PQ = index_pq.search(test_data[0].reshape(1,-1), k=10)

929 µs ± 864 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
#%%timeit
_, closest_PQ = index_pq.search(test_data, k=10)

In [36]:
print(f'smetric@9 = {smetric_9(closest_PQ, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_PQ, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_PQ, closest_baseline, 1):.1f}')

smetric@9 = 42.69
recall@10 = 87.4
recall@1 = 86.9


## CustomPQ

In [75]:
#%%timeit
index_custompq = CustomIndexPQ(train_data.shape[1], 32, init='random', estimator='minibatchKMeans')
index_custompq.train(train_data)
index_custompq.add(train_data)

In [76]:
codes = index_custompq.codes
codes.nbytes / (1024 * 1024) #MB

0.4241943359375

In [77]:
%%timeit
_, closest_PQ = index_custompq.search(test_data[0], k=10)

6.43 ms ± 4.25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
#%%timeit
_, closest_PQ = index_custompq.search(test_data, k=10)

In [79]:
print(f'smetric@9 = {smetric_9(closest_PQ, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_PQ, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_PQ, closest_baseline, 1):.1f}')

smetric@9 = 32.02
recall@10 = 60.3
recall@1 = 68.9


## FAISS IVF

In [12]:
#%%timeit
nlist, nprobe = 256, 32
quantizer = faiss.IndexFlatL2(train_data.shape[1])  # the other index
index = faiss.IndexIVFFlat(quantizer, train_data.shape[1], nlist, faiss.METRIC_L2)
index.nprobe = nprobe

index.train(train_data)
index.add(train_data)

In [57]:
codes = faiss.vector_to_array(index.codes)
codes.nbytes / (1024 * 1024) #MB

AttributeError: 'IndexIVFPQ' object has no attribute 'codes'

In [13]:
%%timeit
_, closest_ivf = index.search(test_data[0].reshape(1,-1), k=10)

113 µs ± 49.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [157]:
#%%timeit
_, closest_ivf = index.search(test_data, k=10)

In [158]:
%%timeit
_, closest_ivf = index.search(test_data, k=10)

10.4 ms ± 81.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [159]:
print(f'smetric@9 = {smetric_9(closest_ivf, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_ivf, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_ivf, closest_baseline, 1):.1f}')

smetric@9 = 44.41
recall@10 = 98.0
recall@1 = 100.0


## Custom IVF

In [11]:
import cProfile

In [27]:
%%timeit
index_customivf = CustomOptimizedIndexIVF(d=train_data.shape[1], nlist=100, nprobe=4, init='random', estimator='KMeans')
index_customivf.train(train_data)
index_customivf.add(train_data)

2.05 s ± 87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
index_customivf = CustomOptimizedIndexIVF(d=train_data.shape[1], nlist=100, nprobe=4, init='random', estimator='KMeans')
index_customivf.train(train_data)
index_customivf.add(train_data)

In [29]:
nbytes = [x.nbytes for x in index_customivf.codes]
sum(nbytes) / (1024 * 1024) #MB

20.361328125

In [30]:
%%timeit
closest_IVF = index_customivf.search(test_data[0], k=10)

573 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%%timeit
closest_IVF = index_customivf.search(test_data, k=10)

45.1 ms ± 70.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
closest_IVF = index_customivf.search(test_data, k=10)

In [33]:
print(f'smetric@9 = {smetric_9(closest_IVF, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_IVF, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_IVF, closest_baseline, 1):.1f}')

smetric@9 = 38.26
recall@10 = 83.1
recall@1 = 85.2


## FAISS IVFPQ

In [42]:
o2.shape

(2,)

In [55]:
o2

array([[0., 1.]])

In [57]:
i2 = np.eye(2)[:1]
o2 = 1-i2
cdist(o2, i2, metric='seuclidean')

array([[2.]])

In [67]:
#%%timeit
nlist, nprobe, m = 1024, 128, 384
quantizer = faiss.IndexFlatL2(train_data.shape[1])  # the other index
index = faiss.IndexIVFPQ(quantizer, train_data.shape[1], nlist, m, 8)
index.nprobe = nprobe

index.train(train_data)
index.add(train_data)

WARNING clustering 13900 points to 1024 centroids: please provide at least 39936 training points


In [70]:
%%timeit
_, closest_ivf = index.search(test_data[0].reshape(1,-1), k=10)

6.09 ms ± 4.76 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%%timeit
_, closest_ivf = index.search(test_data, k=10)

580 µs ± 83.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [68]:
_, closest_ivf = index.search(test_data, k=10)

In [69]:
print(f'smetric@9 = {smetric_9(closest_ivf, closest_baseline):.2f}')
print(f'recall@10 = {recall_n(closest_ivf, closest_baseline, 10):.1f}')
print(f'recall@1 = {recall_n(closest_ivf, closest_baseline, 1):.1f}')

smetric@9 = 44.72
recall@10 = 98.5
recall@1 = 100.0


## TBD

In [25]:
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import euclidean_distances

In [91]:
class CustomIndexIVF:
    """ Custom implementation of IVF Index 
    d: size of original embeddings
    nlist: number of centroids to build
    nprobe: number of closest-neighbor centroids to visit during search
    estimator_kwargs: Additional hyperparameters passed onto the sklearn KMeans/MiniBatchKMeans class
    """

    def __init__(self, d, nlist, nprobe, estimator='KMeans', **estimator_kwargs):
        # self.m = m
        # self.k = 2**8       # nb of clusters per segment
        # self.ds = d // m
        # self.codes = None

        self.d = d
        self.nlist = nlist
        self.nprobe = nprobe

        self.centroids = None
        self.labels = None
        self.codes = None

        if estimator.lower() == 'kmeans':
            self.estimator = KMeans(n_clusters=self.nlist, **estimator_kwargs)
        elif estimator.lower() == 'minibatchkmeans':
            self.estimator = MiniBatchKMeans(n_clusters=self.nlist, **estimator_kwargs)
        else:
            raise ValueError(f"Unknown estimator `{estimator}`. Choose from [`KMeans`, `MiniBatchKMeans`].")

        self.is_trained = False

    
    def train(self, X):
        assert not self.is_trained, "estimator is already trained"

        self.estimator.fit(X)

        self.centroids = self.estimator.cluster_centers_

        self.is_trained = True


    def add(self, X):
        assert self.is_trained, "estimator has to be trained"

        self.codes = X
        self.labels = self.estimator.predict(X)
    
    
    def find_closest_centroids(self, Y):
        assert self.centroids is not None, "need to run `train` first, to train and learn centroids"

        distances_to_centroids = euclidean_distances(Y, self.centroids, squared=True)
        closest_centroids_indices = np.argsort(distances_to_centroids, axis=1)[:, :self.nprobe]
        return closest_centroids_indices


    def aggregate_vectors(self, centroids_indices):
        assert self.codes is not None, "need to run `add` first to learn labels and codes"

        X = []
        indices = []

        for i in range(len(centroids_indices)):
            X_i = np.empty((0, self.d))
            indices_i = []

            for j in range(len(self.codes)):
                if self.labels[j] in centroids_indices[i]:
                    X_i = np.vstack((X_i, self.codes[j, :]))
                    indices_i.append(j)

            X.append(X_i)
            indices.append(indices_i)

        return indices, X


    def search(self, Y, k):
        Y = np.atleast_2d(Y)

        centroids_to_explore = self.find_closest_centroids(Y)

        indices, X = self.aggregate_vectors(centroids_to_explore)

        distances = []
        closest_indices = []

        for i in range(Y.shape[0]):
            distances.append(euclidean_distances(Y[i, :].reshape(1,-1), X[i], squared=True))
            closest_indices.append([indices[i][idx] for idx in np.argsort(distances[i], axis=1)[:, :k][0]])

        closest_indices = np.stack(closest_indices)

        return closest_indices


In [92]:
idx = CustomIndexIVF(d=384, nlist=256, nprobe=10, estimator='KMeans')

In [93]:
idx.train(train_data)
idx.add(train_data)

In [96]:
i=idx.search(train_data[:3], k=10)
i

array([[    0,   218,   222,    16,     5,   234,     2,     4, 11538,
         4603],
       [    1,   219,   233,   234,    13,    15,    16,  5986,  5985,
           12],
       [    2,   220,     0,   234,    16,  4102,   222,   218,  5985,
         4101]])

In [9]:
index_custompq = CustomIndexPQ(train_data.shape[1], 2, init='random', estimator='KMeans')
index_custompq.train(train_data)
index_custompq.add(train_data)

In [10]:
index_custompq.search(test_data[:2],k=4)

(2, 384) (256, 192)
(2, 384) (256, 192)


(array([[0.5182861 , 0.5457897 , 0.55056983, 0.5727137 ],
        [0.64131856, 0.64131856, 0.64131856, 0.64131856]], dtype=float32),
 array([[ 2235, 12181,  9786,  2332],
        [ 7913,  5802,  7064,  6911]]))

In [58]:
arr = np.empty((0, 3))

# Stack new data onto the array
arr = np.vstack((arr, [1, 2, 3]))
arr = np.vstack((arr, [4, 5, 6]))
arr = np.vstack((arr, [7, 8, 9]))

# Print the final array
print(arr)

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
